In [1]:
import numpy as np
import random
from scipy import ndimage, misc
import copy
import time
from deap import base
from deap import creator
from deap import tools
import game


In [2]:
n_weights = 9
population_size = 30
crossover_probability = 1
mutation_probability = 1/n_weights

moves = [0,1,2,3]
directions = {
        0: "down",
        1: "up",
        2: "left",
        3: "right"
    }

generations = 10000
gameIterations = 2000

In [3]:
''' DEAP SETUP PT 1'''

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("GameInstance", game.Game)
creator.create("Individual", list, 
                fitness=creator.FitnessMax, 
                gameInstance=creator.GameInstance)

toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.uniform, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, n_weights)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [4]:
''' Example '''

# Create individual
ind = toolbox.individual()
print(ind) # 10 weights between 0-1

# Game within Individual
print("---")
print(ind.gameInstance.grid)

# Make a move in the game
ind.gameInstance.move(0) # 0 => down 
print("---")
print(ind.gameInstance.grid)

[0.884865167191436, 0.3818097057123495, 0.4747580138882843, 0.7612298756082962, 0.14005419911657102, 0.11517715129681083, 0.05663818571753243, 0.8086528887434529, 0.7472972408072208]
---
[[0 0 2 0]
 [0 0 2 0]
 [0 0 0 0]
 [0 0 0 0]]
---
[[0 0 0 0]
 [0 0 2 0]
 [0 0 0 0]
 [0 0 4 0]]


In [5]:
def f10(newstate, currentstate):

    f1, f2, f3, f4, f5, f6, f7, f8, f9 = 0, 0, 0, 0, 0, 0, 0, 0, 0
    ###
    flatstate = newstate.flatten()
    ###
    maxval= max(flatstate)
    ###
    secondmaxval = max(np.where(flatstate == maxval, 1, flatstate)) 
    secondmaxval = secondmaxval if secondmaxval != 1 else 0
    ###
    thirdmaxval = max(np.where(flatstate == secondmaxval, 1, flatstate)) 
    thirdmaxval = thirdmaxval if thirdmaxval != 1 else 0
    ###
    maxvalues = [maxval, secondmaxval, thirdmaxval]
    ###
    f5cells = [[0,0],[0,3],[1,1],[2,2],[3,0]]
    f6cells = [[2,3],[3,2],[0,1],[0,2],[1,1],[2,0],[1,3],[3,1]]
    ###
    newstateT = newstate.T

    highestValuesIndeces = [0,0]
    for nrow in range(4):

        for ncolumn in range(4):
            val = newstate[nrow,ncolumn]

            if val != 0: f4 += 1  
            if  val == maxval and (nrow,ncolumn) in [(0,0), (0,3), (3,3), (3,0)]: f1 = 1

            if val == secondmaxval: highestValuesIndeces[1] = [nrow,ncolumn]
            if val == maxval: highestValuesIndeces[0] = [nrow,ncolumn]

            if currentstate[nrow,ncolumn] != val: f3 += 1
            if val == maxval:
                # Check above
                if ncolumn-1 >= 0: 
                    nearbyvalue = newstate[nrow,ncolumn-1]
                    if nearbyvalue in maxvalues: f2 = 1
                    if nearbyvalue == val and [nrow, ncolumn] in f5cells: f5 += 1
                    if nearbyvalue == val and [nrow, ncolumn] in f6cells: f6 += 1

                # Check right
                if nrow+1 <= 3:
                    nearbyvalue = newstate[nrow+1,ncolumn]
                    if nearbyvalue in maxvalues: f2 = 1
                    if nearbyvalue == val and [nrow, ncolumn] in f5cells: f5 += 1
                    if nearbyvalue == val and [nrow, ncolumn] in f6cells: f6 += 1
                # Check below
                if ncolumn+1 <= 3:
                    nearbyvalue = newstate[nrow,ncolumn+1] 
                    if nearbyvalue in maxvalues: f2 = 1
                    if nearbyvalue == val and [nrow, ncolumn] in f5cells: f5 += 1
                    if nearbyvalue == val and [nrow, ncolumn] in f6cells: f6 += 1
                # Check left
                if nrow-1 >= 0:
                    nearbyvalue = newstate[nrow-1,ncolumn]
                    if nearbyvalue in maxvalues: f2 = 1
                    if nearbyvalue == val and [nrow, ncolumn] in f5cells: f5 += 1
                    if nearbyvalue == val and [nrow, ncolumn] in f6cells: f6 += 1
    

    # If the highest and second highest value is along the row
    if highestValuesIndeces[0][0] == highestValuesIndeces[1][0] and highestValuesIndeces[0][1] != highestValuesIndeces[1][1]:
        row = newstate[highestValuesIndeces[0][0]]
        if 0 not in newstate: f7 = 1
        if np.array_equiv(np.sort(row), row) or np.array_equiv(np.sort(row)[::-1], row): f8 = 1
        f9 = sum([1 for elem in row if elem != 0])/f4

    # If the highest and second highest value is along the column
    elif highestValuesIndeces[0][1] == highestValuesIndeces[1][1] and highestValuesIndeces[0][0] != highestValuesIndeces[1][0]:
        column = newstate[highestValuesIndeces[0][1]]
        if 0 not in newstateT: f7 = 1
        if np.array_equiv(np.sort(column), column) or np.array_equiv(np.sort(column)[::-1], column): f8 = 1
        f9 = sum([1 for elem in column if elem != 0])/f4
        
    else:
        f8 = 0
        f7 = 0

    return f1,f2,f3,f4,f5,f6,f7,f8,f9


In [6]:
st = ind.gameInstance.project(1)
f10(st, ind.gameInstance.grid)

(0, 1, 4, 2, 0, 0, 0, 0, 0.5)

In [7]:
# The state evaluation function
def stateEvaluation(ind, state):
    f1,f2,f3,f4,f5,f6,f7,f8,f9 = f10(state, ind.gameInstance.grid)
    E = ind[0]*f1 + ind[1]*f2 + ind[2]*f3 + ind[3]*f4 + ind[4]*f5 + ind[5]*f6 + ind[6]*f7 + ind[7]*f8 + ind[8]*f9
    return E

In [8]:
ind = toolbox.individual()

statesEvaluation = []

# Generate possible future states
for move in moves:
    state = ind.gameInstance.project(move)
    E = stateEvaluation(ind, state)
    print(E)
    statesEvaluation.append(E)

chosenMove = moves[np.argmax(statesEvaluation)]


1.9374151558804953
3.2346594422446033
0.9134238978908674
2.93770367468792


In [9]:
# Game Evaluation Function
def gameEvaluation(moves, individual, gameIterations):

    # Play game for 2000 moves
    happy = True
    for iter in range(gameIterations):
        statesEvaluation = []

        # Evaluate all possible moves (projections)
        for move in moves:
            state = ind.gameInstance.project(move)
            if np.array_equal(state, individual.gameInstance.grid): E = 0
            else: E = stateEvaluation(individual, state)
            statesEvaluation.append(E)
            
        # Choose the best possible move with maximum E
        chosenMove = moves[np.argmax(statesEvaluation)]

        try: individual.gameInstance.move(chosenMove)
        except: 
            fitness = (individual.gameInstance.score,)
            individual.gameInstance.start()
            # Return fitness
            return fitness

    # Return the fitness, 
    # Maximum value and number of zeros after 2000 game moves
    fitness = (individual.gameInstance.score,)
    individual.gameInstance.start()
    return (iter,)

In [10]:
def smallIncrementMut(ind, indpb):
    for i in range(len(ind)):
        if random.random() > indpb:
            ind[i] += np.random.normal(0, 0.2, 1)[0]
    return ind,

In [11]:
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=mutation_probability)
#toolbox.register("mutate", tools.mutPolynomialBounded, eta=0.5, low=0, up=10, indpb=0.1)
#toolbox.register("mutate", smallIncrementMut, indpb=0.1)
toolbox.register("mate", tools.cxTwoPoint)
#toolbox.register("mate", tools.cxSimulatedBinary, eta=1.1)
toolbox.register("select", tools.selTournament, k=population_size-1)
toolbox.register("gameEvaluation", gameEvaluation, gameIterations=gameIterations, moves=moves)

In [12]:
def main():
    best = []
    average = []
    # Create the population
    population = toolbox.population(population_size)

    # Evaluate
    for individual in population:
        individual.fitness.values = toolbox.gameEvaluation(individual=individual)

    # Keep track of the no. of generations
    generations = 0
    
    # Begin evolution
    while generations < 1000:
                
        # Selection
        #selected = toolbox.select(population, len(population))
        offspring = [toolbox.clone(ind) for ind in population]
        
        # Crossover
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random()> crossover_probability:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # Mutation
        for mutant in offspring:
            toolbox.mutate(mutant)
            del mutant.fitness.values

        # Evaluate new individuals with no fitness
        for individual in offspring:
            individual.fitness.values = toolbox.gameEvaluation(individual=individual)
        for individual in population:
            individual.fitness.values = ((toolbox.gameEvaluation(individual=individual)[0] + individual.fitness.values[0])/2,)
   
        population = tools.selBest(offspring+population,k=1) + toolbox.select(offspring+population, tournsize=3)
        
        best.append(tools.selBest(population, k=1)[0])
        avg = sum([ind.fitness.values[0] for ind in population])/len(population)
        average.append(average)
        print(generations, best[-1].fitness.values[0], avg, best[-1].gameInstance.highestnumber)
        # Increment the generation counter
        generations += 1    
    
    return tools.selBest(population,k=1), best, average
    

In [13]:
best_ind, best, average = main()

0 792.0 363.06666666666666 64
1 940.0 429.65833333333336 128
2 604.0 423.125 64
3 612.0 381.840625 64
4 1148.0 421.45416666666665 128
5 1340.0 498.42083333333335 128
6 676.0 496.39375 64
7 628.0 403.43365885416665 64
8 1028.0 529.4962565104166 128
9 752.0 449.49583333333334 128
10 624.0 430.9421875 128
11 532.0 432.34765625 64
12 700.0 439.1645833333333 64
13 876.0 441.80390625 128
14 864.0 423.6099446614583 128
15 736.0 384.93723958333334 128
16 512.0 405.29765625 64
17 588.0 372.73229166666664 64
18 864.0 375.1809895833333 128
19 772.0 449.50323079427085 128
20 716.0 463.6166666666667 64
21 896.0 456.19166666666666 128
22 904.0 475.296875 64
23 804.0 442.1901041666667 64
24 718.0 477.22838541666664 64
25 964.0 491.84459228515624 128
26 1596.0 440.8092198689779 128
27 984.0 471.1692708333333 128
28 892.0 494.6685465494792 64
29 964.0 420.8979166666667 128
30 996.0 449.43333333333334 128
31 1260.0 453.10364583333336 128
32 1048.0 353.2990234375 64
33 724.0 418.928125 64
34 1012.0 455.6

In [ ]:
winning_weights = [0.8870351642886638, 1.0, 4.384887169951358, 1.0, 1.0, 3.693619065991971e-06, 0.6856477241443923, -0.16700189754028072, 1.0]

In [ ]:
best_ind = toolbox.individual()
best_ind[::] = winning_weights

witchtrials = []

best_ind.gameInstance.start()
# Play game with best individual
for trial in range(1000):
    witchtrials.append(gameEvaluation(moves, best_ind, 4000)[0])

print(sum(witchtrials)/len(witchtrials))


In [ ]:
min(witchtrials)

In [ ]:
max(witchtrials)

In [ ]:
import matplotlib.pyplot as plt 

#plt.style.use('ggplot')

plt.plot(best_fitnesses)
plt.plot(average)

    


In [ ]:
sum

In [ ]:
ind = [0.8870351642886638, 1.0, 4.384887169951358, 1.0, 1.0, 3.693619065991971e-06, 0.6856477241443923, -0.16700189754028072, 1.0]